## Configuration

In [1]:
starting_config = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"
data_dir = "../../data/models/detectron2/no-augmentations"
output_dir = "runs/train/no-augmentations"
train_data = "train_data"
valid_data = "valid_data"
test_data = "test_data"
final_model_file = "model_final.pth"
n_classes = 1
batch_size_per_image = 16
images_per_batch = 4
learning_rate = 0.001
warmup_iters = 1000
iters = 1500
eval_iters = 300
steps = [] # (1000, 1500)
confidence_threshold = 0.3

# Detectron2 FRCNN Object Detection

## Setup

In [2]:
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger

setup_logger()

register_coco_instances(train_data, {}, f"{data_dir}/train/_annotations.coco.json", f"{data_dir}/train")
register_coco_instances(valid_data, {}, f"{data_dir}/valid/_annotations.coco.json", f"{data_dir}/valid")
register_coco_instances(test_data, {}, f"{data_dir}/test/_annotations.coco.json", f"{data_dir}/test")

## Train

In [3]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    os.makedirs(output_dir, exist_ok=True)
    output_folder = output_dir
    return COCOEvaluator(dataset_name, cfg, False, output_folder)

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(starting_config))
cfg.OUTPUT_DIR = output_dir

cfg.DATASETS.TRAIN = (train_data,)
cfg.DATASETS.TEST = (valid_data,)
cfg.DATALOADER.NUM_WORKERS = 1

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(starting_config)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = batch_size_per_image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = n_classes

cfg.SOLVER.IMS_PER_BATCH = images_per_batch
cfg.SOLVER.BASE_LR = learning_rate
cfg.SOLVER.WARMUP_ITERS = warmup_iters
cfg.SOLVER.MAX_ITER = iters
cfg.SOLVER.STEPS = steps

cfg.TEST.EVAL_PERIOD = eval_iters

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[01/02 15:48:13 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[01/02 15:48:14 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\Gary Blackwood\dev\glasgow-litter\venv\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/02 15:48:45 d2.utils.events]:  eta: 0:33:51  iter: 19  total_loss: 1.559  loss_cls: 0.7676  loss_box_reg: 0  loss_rpn_cls: 0.7577  loss_rpn_loc: 0.06598  time: 1.3440  data_time: 0.1071  lr: 1.9981e-05  max_mem: 7206M
[01/02 15:49:13 d2.utils.events]:  eta: 0:33:53  iter: 39  total_loss: 0.7917  loss_cls: 0.4258  loss_box_reg: 0  loss_rpn_cls: 0.2436  loss_rpn_loc: 0.0771  time: 1.3770  data_time: 0.0369  lr: 3.9961e-05  max_mem: 7206M
[01/02 15:49:41 d2.utils.events]:  eta: 0:34:21  iter: 59  total_loss: 0.263  loss_cls: 0.1765  loss_box_reg: 0  loss_rpn_cls: 0.04035  loss_rpn_loc: 0.04779  time: 1.3944  data_time: 0.0377  lr: 5.9941e-05  max_mem: 7206M
[01/02 15:50:11 d2.utils.events]:  eta: 0:34:03  iter: 79  total_loss: 0.1692  loss_cls: 0.07135  loss_box_reg: 0  loss_rpn_cls: 0.04229  loss_rpn_loc: 0.05305  time: 1.4149  data_time: 0.0374  lr: 7.9921e-05  max_mem: 7206M
[01/02 15:50:41 d2.utils.events]:  eta: 0:33:39  iter: 99  total_loss: 0.1164  loss_cls: 0.03014  loss_box_r

## Evaluate

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

In [4]:
from detectron2.data import MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, final_model_file)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator(test_data, cfg, False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, test_data)
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [01/02 16:40:31 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [01/02 16:40:31 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/02 16:40:31 d2.data.datasets.coco]: Loaded 119 images in COCO format from ../../data/models/detectron2/no-augmentations/test/_annotations.coco.json
[01/02 16:40:31 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|   litter   | 0            |   litter   | 248          |
|            |              |            |              |
|   total    | 248          |            |              |
[01/02 16:40:31 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style=

OrderedDict([('bbox',
              {'AP': 0.0,
               'AP50': 0.0,
               'AP75': 0.0,
               'APs': 0.0,
               'APm': 0.0,
               'APl': 0.0,
               'AP-litter': 0.0})])

## Visualize

In [ ]:
import glob
import cv2
from detectron2.utils.visualizer import Visualizer

cfg.DATASETS.TEST = (test_data, )
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, final_model_file)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold

predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get(test_data)

n_images = 3
title = "Faster RCNN Litter Detection"

for imageName in glob.glob(f"{data_dir}/test/*jpg")[:n_images]:
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1], metadata=test_metadata)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  img = out.get_image()[:, :, ::-1]
  cv2.imshow(title, img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
